# PIK3CA: Effects of Mutation

<b> Standard imports for playing with and plotting data frames. </b>

In [1]:
import pandas as pd
import numpy as np
import scipy.stats
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt

alt.renderers.enable('notebook')


RendererRegistry.enable('notebook')

<b> Import CPTAC </b>

In [2]:
import CPTAC

Loading Dictionary...
Loading Clinical Data...
Loading Proteomics Data...
Loading Transcriptomics Data...
Loading CNA Data...
Loading Phosphoproteomics Data...
Loading Somatic Data...

 ******PLEASE READ******
CPTAC is a community resource project and data are made available
rapidly after generation for community research use. The embargo
allows exploring and utilizing the data, but the data may not be in a
publication until July 1, 2019. Please see
https://proteomics.cancer.gov/data-portal/about/data-use-agreement or
enter CPTAC.embargo() to open the webpage for more details.


In [3]:
somatic_mutations = CPTAC.get_somatic()
proteomics = CPTAC.get_proteomics()
phos = CPTAC.get_phosphoproteomics()

<b> List of proteins that interact with PIK3R1 (according to STRING) </b>

In [4]:
protList = ['IRS1', 'IRS2', 'ATK1', 'P55G', 'ERBB3', 'P85A', 'MRAS', 'PTEN', 'RPS6KB1']

<b> Proteome abundance of proteins that interact with PIK3R1 </b>

In [5]:
pcutoff = 0.05 / len(protList)
gene = 'PIK3CA'

sigResults = [];
for protein in protList: 
    if protein in proteomics.columns:
        cross = CPTAC.compare_mutations(proteomics, protein, gene)
        cross = cross[["Mutation", protein]].dropna(axis=0)
        mutated = cross.loc[cross["Mutation"] == "Missense_Mutation"]
        wt = cross.loc[cross["Mutation"] == "Wildtype"]
        ttest = scipy.stats.ttest_ind(mutated[protein], wt[protein])
        if(ttest[1] <= pcutoff):
            sigResults.append(protein)
            print("Test for " + protein + ": ")
            print(ttest) 

Test for IRS1: 
Ttest_indResult(statistic=-3.985240871803269, pvalue=0.00010769182160893094)
Test for IRS2: 
Ttest_indResult(statistic=-4.5536463563688105, pvalue=1.2342820287857912e-05)
Test for PTEN: 
Ttest_indResult(statistic=-4.26068842042751, pvalue=3.705563596591572e-05)


<b> Print the list of proteins with significant results </b>

In [7]:
print(sigResults)

['IRS1', 'IRS2', 'PTEN']


### Phosphoproteome abundance of interacting proteins

In [8]:
phosProtList = ['IRS1', 'ATK1', 'P55G', 'ERBB3', 'P85A', 'MRAS', 'PTEN', 'RPS6KB1']

In [9]:
sigResults = [];
pcutoff = 0.05/len(phosProtList)
print(pcutoff)

for protein in phosProtList: 
        cross = CPTAC.compare_mutations(phos, protein, gene)
        #print(cross.head())
        for loc in cross.columns:
            #print(loc)
            cross_site = cross[["Mutation", loc]].dropna(axis=0)
            #print(cross_site)
            mutated = cross_site.loc[cross_site["Mutation"] == "Missense_Mutation"]
            wt = cross_site.loc[cross_site["Mutation"] == "Wildtype"]
            ttest = scipy.stats.ttest_ind(mutated[loc], wt[loc])
            if(ttest[1] <= pcutoff):
                sigResults.append(loc)
                print("Test for " + loc + ": ")
                print(ttest)             

0.00625
Test for IRS1-S1005: 
Ttest_indResult(statistic=-3.6363762191848834, pvalue=0.00047303574626612497)
Test for IRS1-S1100: 
Ttest_indResult(statistic=-4.8584238737530345, pvalue=3.840176909315442e-06)
Test for IRS1-S1101: 
Ttest_indResult(statistic=-5.065504281640339, pvalue=1.6006637681340987e-06)
Test for IRS1-S1134: 
Ttest_indResult(statistic=-3.0899242452815208, pvalue=0.0025256893869634324)
Test for IRS1-S270: 
Ttest_indResult(statistic=-3.5576954142473682, pvalue=0.0005101046560626368)
Test for IRS1-S323: 
Ttest_indResult(statistic=-3.4808173935784033, pvalue=0.0006942175734218259)
Test for IRS1-S330: 
Ttest_indResult(statistic=-3.94235522237334, pvalue=0.00015618968862050566)
Test for IRS1-S415: 
Ttest_indResult(statistic=-3.9165469385239104, pvalue=0.00017017081267402305)
Test for IRS1-S421: 
Ttest_indResult(statistic=-3.088740692034591, pvalue=0.002489442917970644)
Test for IRS1-S527: 
Ttest_indResult(statistic=-3.1953305888992727, pvalue=0.001747433169051716)
Test for I

/Users/emilyhosk/anaconda3/envs/research/lib/python3.7/site-packages/scipy/stats/stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


ValueError: Cannot index with multidimensional key